In [48]:
import pandas as pd
import os
from glob import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull

In [2]:
classes = pd.DataFrame(index=[1,2,3,4,5,6,7,8], data ={'name':['Melanoma','Melanocytic nevus',
'Basal cell carcinoma',
 'Actinic keratosis',
 'Benign keratosis',
 'Dermatofibroma', 
 'Vascular lesion',
 'Squamous cell carcinoma']},)
classes

,name
1,Melanoma
2,Melanocytic nevus
3,Basal cell carcinoma
4,Actinic keratosis
5,Benign keratosis
6,Dermatofibroma
7,Vascular lesion
8,Squamous cell carcinoma


In [3]:
data = pd.read_csv('metadataTrainFilled.csv')
print('Number of training images:',data['ID'].count())
print('Missing number of sexes in training set:',data['SEX'].isna().sum())
print('Missing number of ages in training set:',data['AGE'].isna().sum())
print('Missing number of locations in training set:',data['POSITION'].isna().sum())
data.head()

Number of training images: 18998
Missing number of sexes in training set: 0
Missing number of ages in training set: 0
Missing number of locations in training set: 0


,ID,CLASS,SEX,AGE,POSITION
0,ISIC_0000000,2,female,55.0,anterior torso
1,ISIC_0000001,2,female,30.0,anterior torso
2,ISIC_0000002,1,female,60.0,upper extremity
3,ISIC_0000003,2,male,30.0,upper extremity
4,ISIC_0000004,1,male,80.0,posterior torso


In [4]:
test = pd.read_csv('metadataTestFilled.csv')
print('Number of testing images:',test['ID'].count())
print('Missing number of sexes in testing set:',test['SEX'].isna().sum())
print('Missing number of ages in testing set:',test['AGE'].isna().sum())
print('Missing number of locations in testing set:',test['POSITION'].isna().sum())
test.head()

Number of testing images: 6333
Missing number of sexes in testing set: 0
Missing number of ages in testing set: 0
Missing number of locations in testing set: 0


,ID,SEX,AGE,POSITION
0,ISIC_0000006,female,25.0,posterior torso
1,ISIC_0000011,female,35.0,lower extremity
2,ISIC_0000014,male,35.0,posterior torso
3,ISIC_0000018,male,30.0,posterior torso
4,ISIC_0000022,female,55.0,lower extremity


In [10]:
def f(x):
    if x == 'male':
        return 0
    else:
        return 1
    
data['SEX'] = data['SEX'].map(f)
test['SEX'] = test['SEX'].map(f)

agenp = data['AGE'].to_numpy()
#min_age = np.min(agenp)
max_age = np.max(agenp)

def g(x):
    return x/max_age
    
data['AGE'] = data['AGE'].map(g)
test['AGE'] = test['AGE'].map(g)

poss = np.unique(data['POSITION'].to_numpy())
def h(x):
    if x == poss[0]:
        return 0
    elif x == poss[2]:
        return 0.14
    elif x == poss[6]:
        return 0.29
    elif x == poss[1]:
        return 0.43
    elif x == poss[4]:
        return 0.57
    elif x == poss[7]:
        return 0.71
    elif x == poss[3]:
        return 0.86
    elif x == poss[5]:
        return 1
    
data['POSITION'] = data['POSITION'].map(h)
test['POSITION'] = test['POSITION'].map(h)

In [11]:
data.head()

,ID,CLASS,SEX,AGE,POSITION
0,ISIC_0000000,2,1,0.647059,0.00
1,ISIC_0000001,2,1,0.352941,0.00
2,ISIC_0000002,1,1,0.705882,0.71
3,ISIC_0000003,2,0,0.352941,0.71
4,ISIC_0000004,1,0,0.941176,0.29


In [12]:
test.head()

,ID,SEX,AGE,POSITION
0,ISIC_0000006,1,0.294118,0.29
1,ISIC_0000011,1,0.411765,0.86
2,ISIC_0000014,0,0.411765,0.29
3,ISIC_0000018,0,0.352941,0.29
4,ISIC_0000022,1,0.647059,0.86


In [140]:
def seg_features(mask):
    #Area
    area = np.sum(mask)

    if area>2:
        #Perimeter
        cont_test = np.zeros(mask.shape)
        Contour, Hierarchy = cv2.findContours(mask.astype(np.uint8), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        cv2.drawContours(cont_test, Contour, len(Hierarchy[0])-1, 1, 1)
        perim = np.sum(cont_test)
        #Convex hull
        coo = np.where(mask>0)
        x = coo[0]
        y = coo[1]
        xm,ym = np.mean(x),np.mean(y)
        coord = np.array([[a,b] for a,b in zip(x,y)])
        try:
            hull = ConvexHull(coord)
            conv_perim = hull.area
            conv_area = hull.volume
            #Compactness, Roundness, Convexity, Solidity
            compac = 4*np.pi*area/(perim**2)
            roundn = 4*np.pi*area/(conv_perim**2)
            convex = conv_perim/perim
            solid = area/conv_area
        except:
            conv_perim = 0
            conv_area = 0
            compac = 0
            roundn = 0
            convex = 0
            solid = 0
        #Center of bb and aspect ratio
        suml = np.sum(cont_test,axis=0)
        sumc = np.sum(cont_test,axis=1)
        indl = np.where(suml>0)[0]
        indc = np.where(sumc>0)[0]
        x1,x2,y1,y2 = indl[0],indl[-1],indc[0],indc[-1]
        xc,yc = (x1+x2)/2,(y1+y2)/2
        height,width = np.min((x2-x1,y2-y1)),np.max((x2-x1,y2-y1))
        aspectratio = height/width
        dist = np.sqrt((xc-xm)**2+(yc-ym)**2)
    else:
        perim = 0
        aspectratio = 0
        dist = 0
        compac = 0
        roundn = 0
        convex = 0
        solid = 0

    return area,perim,aspectratio,dist,compac,roundn,convex,solid

def channel_features(im,mask):

    b,g,r = cv2.split(im)
    hls = cv2.cvtColor(im,cv2.COLOR_BGR2HLS)
    h,l,_ = cv2.split(hls)

    mil = np.min(l)
    mal = np.max(l)
    avl = np.mean(l)
    val = np.var(l)

    mih = np.min(h)
    mah = np.max(h)
    avh = np.mean(h)
    vah = np.var(h)

    bc = np.mean(b*mask)
    be = np.mean(b*(1-mask))
    gc = np.mean(g*mask)
    ge = np.mean(g*(1-mask))
    rc = np.mean(r*mask)
    re = np.mean(r*(1-mask))
    hc = np.mean(h*mask)
    he = np.mean(h*(1-mask))
    lc = np.mean(l*mask)
    le = np.mean(l*(1-mask))

    diffb = abs(bc-be)
    diffg = abs(gc-ge)
    diffr = abs(rc-re)
    diffh = abs(hc-he)
    diffl = abs(lc-le)


    return mil,mal,avl,val,mih,mah,avh,vah,diffb,diffg,diffr,diffh,diffl

In [158]:
names = data['ID'].to_numpy()
minl,maxl,avgl,varl,minh,maxh,avgh,varh,diffb,diffg,diffr,diffh,diffl = [],[],[],[],[],[],[],[],[],[],[],[],[]
area,perimeter,aspectratio,distance,compactness,roundness,convexity,solidity = [],[],[],[],[],[],[],[]

In [159]:
for i in range(len(names)):
    if i%100 == 0:
        print(i)

    path = './Train/Train/'+names[i]+'.jpg'
    im = cv2.resize(cv2.imread(path),(512,384))
    maskpath = './Train/Modified/'+names[i]+'_seg.png'
    mask = 1/255*cv2.imread(maskpath,cv2.IMREAD_GRAYSCALE)

    mil,mal,avl,val,mih,mah,avh,vah,dib,dig,di_r,dih,dil = channel_features(im,mask)
    ar,perim,asp,dist,compact,roundn,convex,solid = seg_features(mask)
    
    minl.append(mil)
    maxl.append(mal)
    avgl.append(avl)
    varl.append(val)

    minh.append(mih)
    maxh.append(mah)
    avgh.append(avh)
    varh.append(vah)

    diffb.append(dib)
    diffg.append(dig)
    diffr.append(di_r)
    diffh.append(dih)
    diffl.append(dil)

    area.append(ar)
    perimeter.append(perim)
    aspectratio.append(asp)
    distance.append(dist)
    compactness.append(compact)
    roundness.append(roundn)
    convexity.append(convex)
    solidity.append(solid)

minl = (minl-np.min(minl))/(np.max(minl)-np.min(minl))
maxl = (maxl-np.min(maxl))/(np.max(maxl)-np.min(maxl))
avgl = (avgl-np.min(avgl))/(np.max(avgl)-np.min(avgl))
varl = (varl-np.min(varl))/(np.max(varl)-np.min(varl))

minh = (minh-np.min(minh))/(np.max(minh)-np.min(minh))
maxh = (maxh-np.min(maxh))/(np.max(maxh)-np.min(maxh))
avgh = (avgh-np.min(avgh))/(np.max(avgh)-np.min(avgh))
varh = (varh-np.min(varh))/(np.max(varh)-np.min(varh))

diffb = (diffb-np.min(diffb))/(np.max(diffb)-np.min(diffb))
diffg = (diffg-np.min(diffg))/(np.max(diffg)-np.min(diffg))
diffr = (diffr-np.min(diffr))/(np.max(diffr)-np.min(diffr))
diffh = (diffh-np.min(diffh))/(np.max(diffh)-np.min(diffh))
diffl = (diffl-np.min(diffl))/(np.max(diffl)-np.min(diffl))

area = (area-np.min(area))/(np.max(area)-np.min(area))
perimeter = (perimeter-np.min(perimeter))/(np.max(perimeter)-np.min(perimeter))
distance = (distance-np.min(distance))/(np.max(distance)-np.min(distance))

0
100
200
300
400
500


C:\Users\maelm\AppData\Local\Temp\ipykernel_33376\837393457.py:41: RuntimeWarning: invalid value encountered in scalar divide
  aspectratio = height/width


600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18500
18600
18700
18800


In [262]:
id = data['ID'].to_numpy()
clas = data['CLASS'].to_numpy()
sex = data['SEX'].to_numpy()
age = data['AGE'].to_numpy()
pos = data['POSITION'].to_numpy()
compactness = np.minimum(compactness,1)
roundness = np.minimum(roundness,1)
convexity = np.minimum(convexity,1.3)/1.3
solidity = np.minimum(solidity,1)

dic = {'ID':id,'CLASS':clas,'SEX':sex,'AGE':age,'POSITION':pos,'MinL':minl,'MaxL':maxl,'AvgL':avgl,'VarL':varl,'MinH':minh,'MaxH':maxh,'AvgH':avgh,'VarH':varh,'DiffB':diffb,
        'DiffG':diffg,'DiffR':diffr,'DiffH':diffh,'DiffL':diffl,'Area':area,'Perimeter':perimeter,'Aspect Ratio':aspectratio,'Distance':distance,'Compactness':compactness,
        'Roundness':roundness,'Convexity':convexity,'Solidity':solidity}
print(len(dic.keys()))

26


In [263]:
df = pd.DataFrame(dic)
df = df.fillna(0)
df.head()

,ID,CLASS,SEX,AGE,POSITION,MinL,MaxL,AvgL,VarL,MinH,...,DiffH,DiffL,Area,Perimeter,Aspect Ratio,Distance,Compactness,Roundness,Convexity,Solidity
0,ISIC_0000000,2,1,0.647059,0.00,0.109091,0.878205,0.565203,0.417285,0.0,...,0.024649,0.301533,0.475569,0.295352,0.693396,0.134803,0.829122,0.873353,0.749499,0.972363
1,ISIC_0000001,2,1,0.352941,0.00,0.072727,0.865385,0.636809,0.083832,0.0,...,0.672806,0.657611,0.071520,0.134933,0.851351,0.142167,0.597424,0.861772,0.640474,0.904813
2,ISIC_0000002,1,1,0.705882,0.71,0.000000,0.814103,0.617074,0.177336,0.0,...,0.427647,0.502230,0.200806,0.202149,0.996241,0.106458,0.747342,0.754826,0.765408,0.847599
3,ISIC_0000003,2,0,0.352941,0.71,0.206061,0.961538,0.703693,0.301640,0.0,...,0.277054,0.463123,0.341318,0.259620,0.916667,0.175795,0.770138,0.895547,0.713340,0.942955
4,ISIC_0000004,1,0,0.941176,0.29,0.000000,0.974359,0.228720,0.660375,0.0,...,0.195897,0.121372,0.349434,0.217391,0.814590,0.071795,1.000000,0.932791,0.844593,0.987723


In [264]:
df.to_csv('./MetadataTrainModified.csv',index=False)

In [265]:
names = test['ID'].to_numpy()
minl,maxl,avgl,varl,minh,maxh,avgh,varh,diffb,diffg,diffr,diffh,diffl = [],[],[],[],[],[],[],[],[],[],[],[],[]
area,perimeter,aspectratio,distance,compactness,roundness,convexity,solidity = [],[],[],[],[],[],[],[]

In [266]:
for i in range(len(names)):
    if i%100 == 0:
        print(i)

    path = './Test/Test/'+names[i]+'.jpg'
    im = cv2.resize(cv2.imread(path),(512,384))
    maskpath = './Test/Modified/'+names[i]+'_seg.png'
    mask = 1/255*cv2.imread(maskpath,cv2.IMREAD_GRAYSCALE)

    mil,mal,avl,val,mih,mah,avh,vah,dib,dig,di_r,dih,dil = channel_features(im,mask)
    ar,perim,asp,dist,compact,roundn,convex,solid = seg_features(mask)
    
    minl.append(mil)
    maxl.append(mal)
    avgl.append(avl)
    varl.append(val)

    minh.append(mih)
    maxh.append(mah)
    avgh.append(avh)
    varh.append(vah)

    diffb.append(dib)
    diffg.append(dig)
    diffr.append(di_r)
    diffh.append(dih)
    diffl.append(dil)

    area.append(ar)
    perimeter.append(perim)
    aspectratio.append(asp)
    distance.append(dist)
    compactness.append(compact)
    roundness.append(roundn)
    convexity.append(convex)
    solidity.append(solid)

minl = (minl-np.min(minl))/(np.max(minl)-np.min(minl))
maxl = (maxl-np.min(maxl))/(np.max(maxl)-np.min(maxl))
avgl = (avgl-np.min(avgl))/(np.max(avgl)-np.min(avgl))
varl = (varl-np.min(varl))/(np.max(varl)-np.min(varl))

minh = (minh-np.min(minh))/(np.max(minh)-np.min(minh))
maxh = (maxh-np.min(maxh))/(np.max(maxh)-np.min(maxh))
avgh = (avgh-np.min(avgh))/(np.max(avgh)-np.min(avgh))
varh = (varh-np.min(varh))/(np.max(varh)-np.min(varh))

diffb = (diffb-np.min(diffb))/(np.max(diffb)-np.min(diffb))
diffg = (diffg-np.min(diffg))/(np.max(diffg)-np.min(diffg))
diffr = (diffr-np.min(diffr))/(np.max(diffr)-np.min(diffr))
diffh = (diffh-np.min(diffh))/(np.max(diffh)-np.min(diffh))
diffl = (diffl-np.min(diffl))/(np.max(diffl)-np.min(diffl))

area = (area-np.min(area))/(np.max(area)-np.min(area))
perimeter = (perimeter-np.min(perimeter))/(np.max(perimeter)-np.min(perimeter))
distance = (distance-np.min(distance))/(np.max(distance)-np.min(distance))

0
100


C:\Users\maelm\AppData\Local\Temp\ipykernel_33376\837393457.py:41: RuntimeWarning: invalid value encountered in scalar divide
  aspectratio = height/width


200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300


In [289]:
id = test['ID'].to_numpy()
sex = test['SEX'].to_numpy()
age = test['AGE'].to_numpy()
pos = test['POSITION'].to_numpy()
compactness = np.minimum(compactness,1)
roundness = np.minimum(roundness,1)
convexity = np.minimum(convexity,1.3)/1.3
solidity = np.minimum(solidity,1)

dic = {'ID':id,'SEX':sex,'AGE':age,'POSITION':pos,'MinL':minl,'MaxL':maxl,'AvgL':avgl,'VarL':varl,'MinH':minh,'MaxH':maxh,'AvgH':avgh,'VarH':varh,'DiffB':diffb,
        'DiffG':diffg,'DiffR':diffr,'DiffH':diffh,'DiffL':diffl,'Area':area,'Perimeter':perimeter,'Aspect Ratio':aspectratio,'Distance':distance,'Compactness':compactness,
        'Roundness':roundness,'Convexity':convexity,'Solidity':solidity}

In [290]:
df = pd.DataFrame(dic)
df.fillna(0)
df.head()

,ID,SEX,AGE,POSITION,MinL,MaxL,AvgL,VarL,MinH,MaxH,...,DiffH,DiffL,Area,Perimeter,Aspect Ratio,Distance,Compactness,Roundness,Convexity,Solidity
0,ISIC_0000006,1,0.294118,0.29,0.000000,0.754839,0.587001,0.103270,0.0,0.988372,...,0.538090,0.584072,0.095695,0.124134,0.832370,0.115471,0.490080,0.825862,0.592566,0.892031
1,ISIC_0000011,1,0.411765,0.86,0.000000,0.787097,0.532010,0.238288,0.0,0.994186,...,0.505371,0.449342,0.187519,0.144732,0.826772,0.137741,0.706445,0.868107,0.693920,0.920792
2,ISIC_0000014,0,0.411765,0.29,0.000000,0.800000,0.655311,0.067169,0.0,0.994186,...,0.494732,0.398469,0.291932,0.197594,0.743363,0.082348,0.590062,0.829090,0.648940,0.940831
3,ISIC_0000018,0,0.352941,0.29,0.363095,0.600000,0.641963,0.064451,0.0,0.994186,...,0.576864,0.464251,0.245173,0.199234,0.651652,0.110793,0.487424,0.804434,0.598776,0.918776
4,ISIC_0000022,1,0.647059,0.86,0.238095,0.535484,0.559884,0.075246,0.0,0.994186,...,0.415722,0.202708,0.425469,0.187204,0.680590,0.151280,0.958078,0.873483,0.805619,0.955092


In [291]:
df.to_csv('./MetadataTestModified.csv',index=False)